In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql import SparkSession
import tarfile
import os

In [ ]:
# Crear Spark Session
spark = SparkSession.builder.getOrCreate()

In [ ]:
%%time
# Extraer el archivo .tgz
with tarfile.open("./data/raw/yelp_dataset.tar", "r") as tar:
    tar.extractall(path="./data/ini/")

In [ ]:
# Peso del json
file_size = os.path.getsize("./data/ini/yelp_academic_dataset_review.json")
print("Tamaño del archivo:", file_size, "bytes")

In [ ]:
%%time
# Carga los archivos extraídos en un DataFrame
df = spark.read.json("./data/ini/yelp_academic_dataset_review.json")

In [ ]:
%%time
# Guardar el df en parquet
df.write.parquet('review.parquet')

In [ ]:
# Peso del parquet
file_size = os.path.getsize("review.parquet")
print("Tamaño del archivo:", file_size, "bytes")

In [ ]:
# Inferir el esquema inicial
initial_schema = df.schema
initial_schema

In [ ]:
# Definir el nuevo esquema
optimized_schema = StructType([
    StructField("review_id", StringType(), nullable=False),
    StructField("user_id", StringType(), nullable=False),
    StructField("business_id", StringType(), nullable=False),
    StructField("stars", IntegerType(), nullable=False),
    StructField("date", DateType(), nullable=False),
    StructField("text", StringType(), nullable=False),
    StructField("useful", IntegerType(), nullable=False),
    StructField("funny", IntegerType(), nullable=False),
    StructField("cool", IntegerType(), nullable=False)
])

In [ ]:
%%time
# Aplicar el esquema optimizado al DataFrame
df_optimized = spark.read.schema(optimized_schema).json("./data/ini/yelp_academic_dataset_review.json")

In [ ]:
%%time
# Guardar el df_optimized en parquet
df_optimized.write.parquet('review_op.parquet')